In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/guillaume/Python/kt_simul')
import matplotlib.pyplot as plt


import sys
import os
import glob

import numpy as np
import pandas as pd

from IPython.core.display import HTML

from kt_simul.io.parameters_io import ParamTree
from kt_simul.core.simul_spindle import Metaphase
from kt_simul.io.simuio import SimuIO
from kt_simul.utils.progress import pprogress
from kt_simul.core import parameters
from kt_simul.gui.animation import Animator

PARAMFILE = parameters.PARAMFILE
MEASUREFILE = parameters.MEASUREFILE




fig_path = "figs"

In [2]:

from kt_simul.smp_spindle.smp_spindle import SympySpindle, parameters
from sympy.physics.mechanics import mechanics_printing
mechanics_printing(pretty_print=False)  # Shinny

In [3]:
smp_spindle = SympySpindle('test', 3, 3)

In [4]:
smp_spindle.kanesmethod()

In [5]:
A0_args, A0_np = smp_spindle.get_np_A0()

In [6]:
At_np, At_args = smp_spindle.get_np_At()

In [9]:
smp_spindle.B_qq.subs({'F_k': 0})

Matrix([
[0,                                                                                                                                                                                         0,                                                                                                                                                                                         0,                                                                                                                                                                                         0,        0,        0,        0,        0,        0,        0,        0,        0,        0,                                                                                                                                                                                                     0,                                                                                                                                          

In [6]:
from sympy.printing import print_ccode


In [7]:
adim = {parameters['F_k']: 1, parameters['V_k']: 1}
active_params = ['F_k', 'V_k', 'F_mz', 'V_mz']
active_zero = {parameters[name]: 0 for name in active_params}
At = smp_spindle.A_uu.subs(active_zero).subs(adim)


In [9]:
import pickle

In [17]:
from sympy.utilities.codegen import codegen

In [23]:
c_string = codegen(('At', At), 'c')[0][1]

In [12]:
(1 + 3 * 3 * (1 + 3) * 2)**2

5329

In [25]:

print(c_string)

/******************************************************************************
 *                     Code generated with sympy 0.7.6.1                      *
 *                                                                            *
 *              See http://www.sympy.org/ for more information.               *
 *                                                                            *
 *                       This file is part of 'project'                       *
 ******************************************************************************/
#include "At.h"
#include <math.h>

void At(double mu_c, double mu_ch, double mu_k, double mu_s, double *out_210494822497673222) {

   out_210494822497673222[0] = -1.0L/2.0L*mu_s;
   out_210494822497673222[1] = 0;
   out_210494822497673222[2] = 0;
   out_210494822497673222[3] = 0;
   out_210494822497673222[4] = 0;
   out_210494822497673222[5] = 0;
   out_210494822497673222[6] = 0;
   out_210494822497673222[7] = 0;
   out_210494822497673

In [14]:
with open('At_33', 'b+') as At_33: 
    pickle.dump(At, At_33)

ValueError: Must have exactly one of create/read/write/append mode and at most one plus

In [ ]:
paramtree = ParamTree(PARAMFILE)
paramtree.change_dic('dt', 1)
paramtree.change_dic('span', 1000)
paramtree.change_dic('t_A', 750)

paramtree.change_dic('d_alpha', 0.05)  # 0.05
paramtree.change_dic('k_a', 0.06)  # 0.06
paramtree.change_dic('orientation', 1)  # 1

paramtree.change_dic('N', 3)  # 3
paramtree.change_dic('Mk', 3)  # 3
paramtree.change_dic('mus', 500)  # 500
paramtree.change_dic('L0', 0.3)  # 0.3

paramtree.change_dic('ldep', 0)  # 0.4
paramtree.change_dic('Fk', 10)  # 10.0

## Three states parameters
paramtree.change_dic('k_res', 0.02)  # 0.06
paramtree.change_dic('k_cat', 0.06)  # 0.06
paramtree.change_dic('vg', 1e-4)  # 0.03
paramtree.change_dic('vs', 1e-4)  # 0.03

# Fixed parameters
paramtree.change_dic('muco', 0)  # 1000
paramtree.change_dic('muc', 400)  # 400
paramtree.change_dic('muk', 500)  # 500

paramtree.change_dic('Vk', 0.03)  # 0.03
paramtree.change_dic('Fmz', 30)  # 100
paramtree.change_dic('Vmz', 0.03)  # 0.03
paramtree.change_dic('kappa_c', 150)  # 67

measuretree = ParamTree(MEASUREFILE, adimentionalized=False)
measuretree.change_dic('mean_metaph_k_dist', 0.3)  # 0.3
measuretree.change_dic('max_metaph_k_dist', 1.)  # 1.
measuretree.change_dic('obs_d0', 0.1)  # 0.1
measuretree.change_dic('poleward_speed', 0.03)  # 0.03

meta = Metaphase(verbose=True,
                 paramtree=paramtree,
                 measuretree=measuretree,
                 initial_plug='random',
                 keep_same_random_seed=False,
                 force_parameters=[])
meta.simul()

fig = meta.show(lims=None)
params = ('k_res', 'k_cat', 'vg', 'vs')    
title = ', '.join('{0}: {1:.3f}'.format(p, paramtree.absolute_dic[p]) for p in params)

ax = fig.gca()
ax.set_title(title)



In [2]:
fig, ax = plt.subplots()
ps_correlations = np.zeros((meta.KD.params['N'], 2,meta.KD.num_steps))

for n, (ch, color) in enumerate(zip(meta.KD.chromosomes, meta.chrom_colors)):
    
    for ps0 in ch.cen_A.plugsites:
        for ps1 in ch.cen_A.plugsites:
            if ps1 != ps0:
                ps_correlations[n, 0, :] += (ps0.mt_state_hist == ps1.mt_state_hist)# & (ps0.state_hist == ps1.state_hist)
    for ps0 in ch.cen_B.plugsites:
        for ps1 in ch.cen_B.plugsites:
            if ps1 != ps0:
                ps_correlations[n, 1, :] += (ps0.mt_state_hist == ps1.mt_state_hist)# & (ps0.state_hist == ps1.state_hist)

    ax.plot(ps_correlations[n, ...].mean(axis=0)/2., c=color, lw=2, alpha=0.3)
ax.plot(ps_correlations.mean(axis=0).mean(axis=0)/2, 'k-')

params = ('k_res', 'k_cat', 'vg', 'vs')    
title = ', '.join('{0}: {1:.3f}'.format(p, paramtree.absolute_dic[p]) for p in params)
ax.set_ylim(-0.1, 3.1)

ax.set_title(title)

NameError: name 'meta' is not defined

In [25]:
meta.KD.params['dt']

1

In [11]:
fig, (ax_cat, ax_res) = plt.subplots(2, 1)


for n, (ch, color) in enumerate(zip(meta.KD.chromosomes, meta.chrom_colors)):
    
    for ps in ch.cen_A.plugsites:
        speed = ps.speed_hist[ps.state_hist != 0]
        k_cat = - np.log(1 - ps.pcat_hist)[ps.state_hist != 0]
        k_res = - np.log(1 - ps.pres_hist)[ps.state_hist != 0]
        ax_cat.plot(speed, k_cat, 'o', c=color, alpha=0.2, label='cen A')
        ax_res.plot(speed, k_res, 'o', c=color, alpha=0.2)
    for ps in ch.cen_B.plugsites:
        speed = ps.speed_hist[ps.state_hist != 0]
        k_cat = - np.log(1 - ps.pcat_hist)[ps.state_hist != 0]
        k_res = - np.log(1 - ps.pres_hist)[ps.state_hist != 0]
        ax_cat.plot(speed, k_cat, 'v', c=color, alpha=0.2, label='cen B')
        ax_res.plot(speed, k_res, 'v', c=color, alpha=0.2)
ax_cat.set_ylabel('k_cat Catastrophe')
ax_res.set_ylabel('k_res Rescue')


In [10]:
fig = plt.figure()
h = plt.hist(k_cat[ps.state_hist != 0], bins=100)

In [5]:
print(title)
print('mean: {:.3f}, std: {:.3f}'.format(ps_correlations.mean()/2, ps_correlations.std()/2))

k_res: 0.600, k_cat: 0.000, vg: 100.000, vs: 100.000
mean: 2.954, std: 0.301


In [6]:
ps0.probas()

(0.25918105244636536, 0.0)

In [24]:


ax.plot(ps_correlations.mean(axis=0).mean(axis=0)/2)